In [519]:
import pandas as pd
import numpy as np

In [520]:
df = pd.read_csv('fundacurrentlistings20240517_dataset.csv')

In [521]:
df = df.drop_duplicates()
df = df.drop(columns={'size', 'url', 'descrip', 'city', 'last_ask_price', 'insulation', 'heating', 'num_of_bathrooms'})
df = df.rename(columns={'ownership':'outside_space', 'building_type':'newbuild'})

# clean the price
df['price'] = df['price'].str.replace('€ ', '')
df['price'] = df['price'].str.replace(' /mnd', '')
df['price'] = df['price'].str.replace('k', '')
df['price'] = df['price'].str.replace('na', '0')
df['price'] = df['price'].str.replace('Huurprijs op aanvraag', '0')
df['price'] = df['price'].str.replace('.', '')

# drop all rows where dependent variable price is equal to zero and higher than 12500 euros a months. Those are buy listings listed wrongly
df = df.drop(df[df['price'] == '0'].index)
df = df.drop(df[pd.to_numeric(df['price']) > 12500].index)
df = df.drop(df[pd.to_numeric(df['price']) < 500].index)


# clean zip_code. Should be len = 6 with 4 numbers and 2 letters
df['zip_code'] = df['zip_code'].str[:7]
df['zip_code'] = df['zip_code'].str.replace(' ', '')

# remove non-houses
df = df.drop(df[df['kind_of_house'] == 'Garage'].index)
df = df.drop(df[df['kind_of_house'] == 'Inpandige garage'].index)
df = df.drop(df[df['kind_of_house'] == 'parkeerkelder'].index)
df = df.drop(df[df['kind_of_house'] == 'Parkeerplaats'].index)

# clean the year. We changed 'Voor XXXX', 'Na XXXX' and 'XXXX-YYYY' to 'XXXX'
df['year'] = df['year'].str.replace('Voor ', '')
df['year'] = df['year'].str.replace('Na ', '')
df['year'] = df['year'].str[:4]

def check_year_validity(label):
    if 0 < label < 2025:
        return label
    else:
        return np.nan

df['year'] = df['year'].astype(int).apply(check_year_validity)

def check_living_area_validity(label):
    if label != 'na':
        return label
    else:
        return np.nan

# clean living_area
df['living_area'] = df['living_area'].str.replace(' m²', '')
df['living_area'] = df['living_area'].astype(str).apply(check_living_area_validity)

df.describe()

,year
count,887.000000
mean,1959.910936
std,91.418085
min,1005.000000
25%,1906.000000
50%,1998.000000
75%,2024.000000
max,2024.000000


In [522]:
df.head(2)

,price,address,zip_code,year,living_area,kind_of_house,newbuild,num_of_rooms,layout,energy_label,outside_space,parking
0,1045,Schipluidenlaan 254,1062HE,2024,44,Portiekflat,Nieuwbouw,2 kamers (1 slaapkamer),Aantal kamers2 kamers (1 slaapkamer)Aantal woo...,na,na,na
1,1068,Krijn Taconiskade 299,1087HW,2022,44,Portiekflat (appartement met open portiek),Nieuwbouw,1 kamer (1 slaapkamer),Aantal kamers1 kamer (1 slaapkamer)Aantal woon...,A++,na,na


In [523]:
# extract number of rooms in total
df['num_of_rooms'] = df['layout'].str.extract(r'Aantal kamers(\d+)')
# extract the number of bathrooms
df['num_of_bedrooms'] = df['layout'].str.extract(r'(\d+) slaapkamer')
# extract the number of bedrooms
df['num_of_bathrooms'] = df['layout'].str.extract(r'(\d+) badkamer')
# extract the number of seperate toilets
df['num_of_separate_toilets'] = df['layout'].str.extract(r'(\d+) apart')

# extract the number of floors
# df['total_num_of_floors'] = df['layout'].str.extract(r'Aantal woonlagen(\d+)') - drop this cause this is not correct in alot of the entry, so I'll delete it

# extract the floor number of house
df['floor_of_house'] = df['layout'].str.extract(r'Gelegen op(\w+)')
df['floor_of_house'] = df['floor_of_house'].str.replace('e', '')

df = df.drop(columns={'layout'})


# parking
df['parking'] = df['parking'].apply(lambda x: 1 if 'Soort parkeergelegenheid' in x else 0)

# dummy newbuild
df['newbuild'] = df['newbuild'].apply(lambda x: 1 if 'Nieuwbouw' in x else 0)

# Turn the energy labels into ordinal values with NaN if value is missing
def check_energy_label(label):
    if 'A++++' in label:
        return 8
    elif 'A+++' in label:
        return 7
    elif 'A++' in label:
        return 6
    elif 'A+' in label:
        return 5
    elif 'A' in label:
        return 4
    elif 'B' in label:
        return 3
    elif 'C' in label:
        return 2
    elif 'D' in label:
        return 1
    else:
        return np.nan

df['energy_label'] = df['energy_label'].astype(str).apply(check_energy_label)

def categorize_housing_type(housing_type):
    apartments_flats = [
        'Portiekflat', 'Portiekflat (appartement met open portiek)', 'Galerijflat',
        'Galerijflat (appartement)', 'Portiekflat (appartement)', 'Tussenverdieping',
        'Bovenwoning (appartement)', 'Portiekwoning (appartement)', 'Benedenwoning (appartement)',
        'Maisonnette', 'Tussenverdieping (appartement)', 'Bovenwoning (appartement met open portiek)',
        'Benedenwoning (souterrain)', 'Benedenwoning (souterrain met open portiek)', 'Dubbel benedenhuis',
        'Maisonnette (appartement)', 'Dubbel benedenhuis (appartement)', 'Bovenwoning (dubbel bovenhuis)',
        'Benedenwoning (appartement met open portiek)', 'Tussenverdieping (appartement met open portiek)',
        'Bovenwoning (open portiek)', 'Beneden + bovenwoning (appartement met open portiek)',
        'Tussenverdieping (open portiek)', 'Beneden + bovenwoning (bel-etage)', 'Benedenwoning (bel-etage)',
        'Penthouse (appartement)', 'Dubbel benedenhuis (bel-etage)', 'Dubbel benedenhuis (appartement met open portiek)',
        'Tussenverdieping (service flat)'
    ]
    single_family_homes = [
        'Eengezinswoning, tussenwoning', 'Eengezinswoning, tussenwoning (hofjeswoning)', 
        'Eengezinswoning, geschakelde woning', 'Eengezinswoning, hoekwoning', 
        'Eengezinswoning, tussenwoning (split-level woning)', 'Eengezinswoning, tussenwoning (drive-in woning)', 
        'Eengezinswoning, 2-onder-1-kapwoning'
    ]
    duplex_multi_level_homes = [
        'Beneden + bovenwoning', 'Benedenwoning (dubbel bovenhuis)', 'Maisonnette (dubbel bovenhuis)', 
        'Dubbel benedenhuis (dubbel bovenhuis met open portiek)', 'Bovenwoning (dubbel bovenhuis met open portiek)', 
        'Beneden + bovenwoning (appartement)'
    ]
    luxury_homes = [
        'Penthouse', 'Herenhuis, tussenwoning', 'Villa, halfvrijstaande woning (waterwoning)', 
        'Landhuis, 2-onder-1-kapwoning', 'Grachtenpand, tussenwoning'
    ]
    waterfront_homes = ['Woonboot, vrijstaande woning (waterwoning)']

    if housing_type in apartments_flats:
        return 'apartments/flats'
    elif housing_type in single_family_homes:
        return 'single-family homes'
    elif housing_type in duplex_multi_level_homes:
        return 'duplex/multi-level homes'
    elif housing_type in luxury_homes:
        return 'luxury homes'
    elif housing_type in waterfront_homes:
        return 'waterfront homes'
    else:
        return np.nan

# Apply the function to the DataFrame
df['kind_of_house'] = df['kind_of_house'].apply(categorize_housing_type)

df.head(200)

,price,address,zip_code,year,living_area,kind_of_house,newbuild,num_of_rooms,energy_label,outside_space,parking,num_of_bedrooms,num_of_bathrooms,num_of_separate_toilets,floor_of_house
0,1045,Schipluidenlaan 254,1062HE,2024,44,apartments/flats,1,2,NaN,na,0,1,NaN,NaN,1
1,1068,Krijn Taconiskade 299,1087HW,2022,44,apartments/flats,1,1,6.0,na,0,1,NaN,NaN,3
2,1099,Haarlemmerweg,1014BL,2024,50,apartments/flats,1,2,NaN,Nee,0,1,NaN,NaN,NaN
3,1103,Krijn Taconiskade 441,1087HW,2022,44,apartments/flats,1,1,6.0,na,0,1,NaN,NaN,5
4,1105,Willem Frogerstraat 41,1062HZ,2024,56,apartments/flats,1,2,NaN,na,0,1,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1700,Plesostraat 123,1043BV,2024,73,apartments/flats,1,3,5.0,na,0,2,NaN,NaN,5
196,1705,Plesostraat 335,1043BV,2024,70,apartments/flats,1,3,5.0,na,0,2,NaN,NaN,18
197,1705,Plesostraat 333,1043BV,2024,70,apartments/flats,1,3,5.0,na,0,2,NaN,NaN,18
198,1710,Plesostraat 61,1043BV,2024,73,apartments/flats,1,3,5.0,na,0,2,NaN,NaN,3


In [524]:
df.to_csv('v8.csv', index=False)
# df.sort_values('total_num_of_floors', ascending=False, key=pd.to_numeric)

In [525]:
# df['kind_of_house'] = df['kind_of_house'].str.casefold()
# df['kind_of_house'].unique().tolist()

dummies = pd.get_dummies(df['kind_of_house'], prefix='type_of_home')

# Concatenate the new dummy columns to the existing DataFrame
df = pd.concat([df, dummies], axis=1)

# Drop the original 'kind_of_house' column if it's no longer needed
df.drop('kind_of_house', axis=1, inplace=True)

In [526]:
df.head(20)

,price,address,zip_code,year,living_area,newbuild,num_of_rooms,energy_label,outside_space,parking,num_of_bedrooms,num_of_bathrooms,num_of_separate_toilets,floor_of_house,type_of_home_apartments/flats,type_of_home_duplex/multi-level homes,type_of_home_luxury homes,type_of_home_single-family homes,type_of_home_waterfront homes
0,1045,Schipluidenlaan 254,1062HE,2024,44,1,2,NaN,na,0,1,NaN,NaN,1,True,False,False,False,False
1,1068,Krijn Taconiskade 299,1087HW,2022,44,1,1,6.0,na,0,1,NaN,NaN,3,True,False,False,False,False
2,1099,Haarlemmerweg,1014BL,2024,50,1,2,NaN,Nee,0,1,NaN,NaN,NaN,True,False,False,False,False
3,1103,Krijn Taconiskade 441,1087HW,2022,44,1,1,6.0,na,0,1,NaN,NaN,5,True,False,False,False,False
4,1105,Willem Frogerstraat 41,1062HZ,2024,56,1,2,NaN,na,0,1,NaN,NaN,1,True,False,False,False,False
5,1115,Schipluidenlaan 270,1062HE,2024,50,1,2,NaN,na,0,1,NaN,NaN,1,True,False,False,False,False
6,1120,Staalmeesterslaan 377,1057PG,1971,58,0,2,5.0,na,0,1,NaN,NaN,13,True,False,False,False,False
7,1120,Staalmeesterslaan 380,1057PG,1971,58,0,2,5.0,na,0,1,NaN,NaN,14,True,False,False,False,False
8,1130,Olga de Haasstraat 513,1095PG,2024,58,1,2,NaN,na,0,1,NaN,NaN,28,True,False,False,False,False
9,1135,Willem Frogerstraat 53,1062HZ,2024,80,1,4,NaN,na,0,3,NaN,NaN,1,True,False,False,False,False
